# Instruction Tests

In [3]:
import serial
import time
import pyvisa 
import numpy as np
from IPython.display import Image, display
from KeysightScope import KeysightScope
from BinaryRoutines import BinaryRoutines

# You will have to change this to whatever COM port the pico is assigned when
# you plug it in.
# On Windows you can open device manager and look at the 'Ports (COM & LPT)' dropdown
# the pico will show up as 'USB Serial Device'
SWEEP_PORT = 'COM3'
PRAWN_PORT = 'COM7'

MHZ = 1000000

sweep_conn = None
prawn_conn = None
sweep_conn = serial.Serial(SWEEP_PORT, baudrate = 152000, timeout = 0.1)
prawn_conn = serial.Serial(PRAWN_PORT, baudrate = 152000, timeout = 0.1)

## Helpers

In [4]:
def sendp(command: str, echo = True) -> str:
    # pico is expecting a newline to end every command
    if command[-1] != '\n':
        command += '\n'

    resp = ''
    try:
        prawn_conn.write(command.encode())
        if echo:
            resp = prawn_conn.readlines()
            resp = "".join([s.decode() for s in resp])

    except Exception as e:
        print("Encountered Error: ", e)

    return resp

In [5]:
## slightly different logic than the ad9959.c funcs due to python vs c
def get_ftw(freq_out: float, freq_sys: float = 500 * MHZ):
    ftw = (freq_out / freq_sys) * 2**32
    return(int(ftw))

def get_pow(phase: float):
    pow = round(phase / 360 * (2**14 - 1))
    return pow

def get_asf(amp: float):
    asf = round(amp * 1024)
    return asf

In [ ]:
f1 = get_ftw(freq_out = 100 * MHZ)
f2 = get_ftw(freq_out = 120 * MHZ)
f3 = get_ftw(freq_out = 110 * MHZ)

a1 = 1023
a2 = 500

p0 = 0
p1 = get_pow(phase = 90)
p2 = get_pow(phase = 180)
p3 = get_pow(phase = 270)

t = 2
d = 1
r = 1


## 243 KB Results
| Table Mode                  | 1 Channel | 2 Channel | 3 Channel | 4 Channel |
|-----------------------------|-----------|-----------|-----------|-----------|
| Single Stepping (Ext Timer) | 16587     | 8579      | 5786      | 4365      |
| Sweep Mode (Ext Timer)      | 8579      | 4364      | 2927      | 2201      |
| Single Stepping (Int Timer) | 5000      | 5000      | 5000      | 4014      |
| Sweep Mode (Int Timer)      | 5000      | 4014      | 2691      | 2024      |

## General Single Step Test

In [7]:
#########################################################
## Change these per every test, rest should be handled ##
#########################################################

num_chans = 3
timing_mode = 1 # 0 for external, 1 for internal
sweep_mode = 0

## 240 Kb results
# n_1_step_ext = 16382
# n_2_step_ext = 8473
# n_3_step_ext = 5714
# n_4_step_ext = 4311

## 243 Kb Results
n_1_step_ext = 16587
n_2_step_ext = 8579
n_3_step_ext = 5786
n_4_step_ext = 4365

n_1_step_int = 5000
n_2_step_int = 5000
n_3_step_int = 5000
n_4_step_int = 4014

#########################################################

ext_timing_template = [
    ('frequency0', '<u4'), 
    ('amplitude0', '<u2'), 
    ('phase0', '<u2'),
    ('frequency1', '<u4'), 
    ('amplitude1', '<u2'), 
    ('phase1', '<u2'),
    ('frequency2', '<u4'), 
    ('amplitude2', '<u2'), 
    ('phase2', '<u2'),
    ('frequency3', '<u4'), 
    ('amplitude3', '<u2'), 
    ('phase3', '<u2')
    ]

ext_timing_dtypes = np.dtype(ext_timing_template[:num_chans*3])

ext_timing_sequence = (f2, a1, p1) # Freq, amp, phase (no time)

int_timing_template = [
    ('frequency0', '<u4'), 
    ('amplitude0', '<u2'), 
    ('phase0', '<u2'),
    ('time0', '<u4'),
    ('frequency1', '<u4'), 
    ('amplitude1', '<u2'), 
    ('phase1', '<u2'),
    ('time1', '<u4'),
    ('frequency2', '<u4'), 
    ('amplitude2', '<u2'), 
    ('phase2', '<u2'),
    ('time2', '<u4'),
    ('frequency3', '<u4'), 
    ('amplitude3', '<u2'), 
    ('phase3', '<u2'),
    ('time3', '<u4')
    ]

int_timing_dtypes = np.dtype(int_timing_template[:num_chans*4])

int_timing_sequence = (f2, a1, p1, t) # Freq, amp, phase, (time)

# tuple addition logic, multiplied by number of instructions
arr_1_ext = [ext_timing_sequence] * n_1_step_ext
arr_2_ext = [ext_timing_sequence + ext_timing_sequence] * n_2_step_ext
arr_3_ext = [ext_timing_sequence + ext_timing_sequence + ext_timing_sequence] * n_3_step_ext
arr_4_ext = [ext_timing_sequence + ext_timing_sequence + ext_timing_sequence + ext_timing_sequence] * n_4_step_ext

arr_1_int = [int_timing_sequence] * n_1_step_int
arr_2_int = [int_timing_sequence + int_timing_sequence] * n_2_step_int
arr_3_int = [int_timing_sequence + int_timing_sequence + int_timing_sequence] * n_3_step_int
arr_4_int = [int_timing_sequence + int_timing_sequence + int_timing_sequence + int_timing_sequence] * n_4_step_int

# combine for easy indexing
list_of_arrs_ext = [arr_1_ext, arr_2_ext, arr_3_ext, arr_4_ext]
list_of_arrs_int = [arr_1_int, arr_2_int, arr_3_int, arr_4_int]

if timing_mode == 0: # External Timing
    gen_instructions = np.array(list_of_arrs_ext[num_chans-1], dtype = ext_timing_dtypes)
    general_test = BinaryRoutines(sweep_conn)
    general_test.set_mode(sweep_mode=sweep_mode, timing_mode=timing_mode, num_channels=num_chans)
    general_test.allocate(start_address=0, instructions=gen_instructions)
    general_test.table_to_memory(instructions=gen_instructions)
    general_test.end_table(instructions=gen_instructions)
    general_test.start_routine(hwstart=True)

    time.sleep(1)

    sendp(f'set 0 0 4000 20000\r\n')
    print(sendp('start\r\n', echo=True))

    time.sleep(2)

else: # timing_mode == 1, internal timing
    gen_instructions = np.array(list_of_arrs_int[num_chans-1], dtype = int_timing_dtypes)
    general_test = BinaryRoutines(sweep_conn)
    general_test.set_mode(sweep_mode=sweep_mode, timing_mode=timing_mode, num_channels=num_chans)
    general_test.allocate(start_address=0, instructions=gen_instructions)
    general_test.table_to_memory(instructions=gen_instructions)
    general_test.end_table(instructions=gen_instructions)
    general_test.start_routine(hwstart=False)
    time.sleep(1)

general_test.send('numtriggers')

sending commands to 3 channels
Got response: ready for 180000 bytes

table written to memory
table end command sent
table executed
5000


## General Sweep Test

In [9]:
#########################################################
## Change these per every test, rest should be handled ##
#########################################################

num_chans = 1
timing_mode = 0 # 0 for external, 1 for internal
sweep_mode = 1

## 240 Kb results
# n_1_sweep_ext = 8472
# n_2_sweep_ext = 4311
# n_3_sweep_ext = 2890
# n_4_sweep_ext = 2174

# n_1_sweep_int = 4998
# n_2_sweep_int = 3960
# n_3_sweep_int = 2653
# n_4_sweep_int = 1995

## 243 Kb Results
n_1_sweep_ext = 8579
n_2_sweep_ext = 4364
n_3_sweep_ext = 2927
n_4_sweep_ext = 2201

n_1_sweep_int = 5000
n_2_sweep_int = 4014
n_3_sweep_int = 2691
n_4_sweep_int = 2024

#########################################################

ext_timing_template = [
    ('start_amplitude0', '<u2'),
    ('stop_amplitude0', '<u2'),
    ('delta0', '<u2'),
    ('rate0', '<u1'),
    ('start_amplitude1', '<u2'),
    ('stop_amplitude1', '<u2'),
    ('delta1', '<u2'),
    ('rate1', '<u1'),
    ('start_amplitude2', '<u2'),
    ('stop_amplitude2', '<u2'),
    ('delta2', '<u2'),
    ('rate2', '<u1'),
    ('start_amplitude3', '<u2'),
    ('stop_amplitude3', '<u2'),
    ('delta3', '<u2'),
    ('rate3', '<u1')
    ]

ext_timing_dtypes = np.dtype(ext_timing_template[:num_chans*4])

ext_timing_sequence = (a1, a2, d, r) # Freq, amp, phase (no time)

int_timing_template = [
    ('start_amplitude0', '<u2'),
    ('stop_amplitude0', '<u2'),
    ('delta0', '<u2'),
    ('rate0', '<u1'),
    ('time0', '<u4'),
    ('start_amplitude1', '<u2'),
    ('stop_amplitude1', '<u2'),
    ('delta1', '<u2'),
    ('rate1', '<u1'),
    ('time1', '<u4'),
    ('start_amplitude2', '<u2'),
    ('stop_amplitude2', '<u2'),
    ('delta2', '<u2'),
    ('rate2', '<u1'),
    ('time2', '<u4'),
    ('start_amplitude3', '<u2'),
    ('stop_amplitude3', '<u2'),
    ('delta3', '<u2'),
    ('rate3', '<u1'),
    ('time3', '<u4')
    ]
int_timing_dtypes = np.dtype(int_timing_template[:num_chans*5])

int_timing_sequence = (a1, a2, d, r, t) # Freq, amp, phase, (time)

# tuple addition logic, multiplied by number of instructions
arr_1_ext = [ext_timing_sequence] * n_1_sweep_ext
arr_2_ext = [ext_timing_sequence + ext_timing_sequence] * n_2_sweep_ext
arr_3_ext = [ext_timing_sequence + ext_timing_sequence + ext_timing_sequence] * n_3_sweep_ext
arr_4_ext = [ext_timing_sequence + ext_timing_sequence + ext_timing_sequence + ext_timing_sequence] * n_4_sweep_ext

arr_1_int = [int_timing_sequence] * n_1_sweep_int
arr_2_int = [int_timing_sequence + int_timing_sequence] * n_2_sweep_int
arr_3_int = [int_timing_sequence + int_timing_sequence + int_timing_sequence] * n_3_sweep_int
arr_4_int = [int_timing_sequence + int_timing_sequence + int_timing_sequence + int_timing_sequence] * n_4_sweep_int

# combine for easy indexing
list_of_arrs_ext = [arr_1_ext, arr_2_ext, arr_3_ext, arr_4_ext]
list_of_arrs_int = [arr_1_int, arr_2_int, arr_3_int, arr_4_int]

if timing_mode == 0: # External Timing
    gen_instructions = np.array(list_of_arrs_ext[num_chans-1], dtype = ext_timing_dtypes)
    general_test = BinaryRoutines(sweep_conn)
    general_test.set_mode(sweep_mode=sweep_mode, timing_mode=timing_mode, num_channels=num_chans)
    general_test.allocate(start_address=0, instructions=gen_instructions)
    general_test.table_to_memory(instructions=gen_instructions)
    general_test.end_table(instructions=gen_instructions)
    general_test.start_routine(hwstart=True)

    time.sleep(1)

    sendp(f'set 0 0 4000 20000\r\n')
    print(sendp('start\r\n', echo=True))

    time.sleep(2)

else: # timing_mode == 1, internal timing
    gen_instructions = np.array(list_of_arrs_int[num_chans-1], dtype = int_timing_dtypes)
    general_test = BinaryRoutines(sweep_conn)
    general_test.set_mode(sweep_mode=sweep_mode, timing_mode=timing_mode, num_channels=num_chans)
    general_test.allocate(start_address=0, instructions=gen_instructions)
    general_test.table_to_memory(instructions=gen_instructions)
    general_test.end_table(instructions=gen_instructions)
    general_test.start_routine(hwstart=False)
    time.sleep(1)

general_test.send('numtriggers')

sending commands to 1 channels
Got response: ready for 60053 bytes

table written to memory
table end command sent
table executed
ok

8579
